# INTELIGENCIA ARTIFICIAL
## Tarea 1: implementacion de gradiente descendiente
### Hecho por: Juan Sebastián Clavijo Martínez<br />
TEMA: Usar el dataset sobre finca raiz en taiwan para hacer una implementación de gradiente descendiente<br />
Fecha: 11-08-2024<br />
Notas: Nada por ahora<br />
**Pontificia Universidad Javeriana**

1. preprocesamiento de datos

In [6]:
# Importado de bibliotecas pertinentes
# se usa el #type: ignore para que mi vscode genere warnings sobre la resolucion de dependencias de los pkts

import pandas as pd  # type: ignore # tratamiento de datos en un dataframe
import numpy as np  # type: ignore # numerical python
import seaborn as sns  # type: ignore # biblioteca versatil para estadistica y visualizacion
import matplotlib.pyplot as plt  # type: ignore # biblioteca para vizualizacion de datos
import ipywidgets as widgets  # https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html#file-upload
from IPython.display import display
from ucimlrepo import fetch_ucirepo  # type: ignore # se traen los datos del repositorio, para mas info revisar: https://github.com/uci-ml-repo/ucimlrepo (el dataset tiene el id=447)

In [7]:
# Parte 1.1: carga de datos
# se trae el dataset
url = 'https://archive.ics.uci.edu/static/public/477/data.csv'
# se hace el obj dataframe
df_rev = pd.read_csv(url)
# se presentan los primeros 5 renglones
df_rev.head(5)

,No,X1 transaction date,X2 house age,X3 distance to the nearest MRT station,X4 number of convenience stores,X5 latitude,X6 longitude,Y house price of unit area
0,1,2012.917,32.0,84.87882,10,24.98298,121.54024,37.9
1,2,2012.917,19.5,306.59470,9,24.98034,121.53951,42.2
2,3,2013.583,13.3,561.98450,5,24.98746,121.54391,47.3
3,4,2013.500,13.3,561.98450,5,24.98746,121.54391,54.8
4,5,2012.833,5.0,390.56840,5,24.97937,121.54245,43.1


2. Implementación de Gradient Booster

In [8]:
# Definimos la función que será graficada
def plot_line(slope):
    x = np.linspace(-10, 10, 100)
    y = slope * x  # Modificamos la pendiente de la línea
    fig, ax = plt.subplots()
    ax.plot(x, y, label=f"y = {slope}x")
    ax.set_ylim(-50, 50)  # Ajustamos los límites de la gráfica
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_title("Gráfica de línea con pendiente ajustable")
    ax.legend()
    plt.show()


# Creamos un slider para ajustar la pendiente
slope_slider = widgets.IntSlider(
    value=1, min=-5, max=5, step=1, description="Pendiente", continuous_update=False
)

# Mostramos el slider y la gráfica
widgets.interactive(plot_line, slope=slope_slider)

interactive(children=(IntSlider(value=1, continuous_update=False, description='Pendiente', max=5, min=-5), Out…